In [10]:
from clickhouse_driver import Client
import pandas as pd

client = Client(host='localhost', database='metro')
query = "SELECT * FROM station"
data = client.execute(query)
df = pd.DataFrame(data, columns=['C/A', 'Unit', 'SCP', 'Station', 'Date', 'Time', 'Description', 'Entries', 'Exits', 'datetime','turnstile'])
df.head()

,C/A,Unit,SCP,Station,Date,Time,Description,Entries,Exits,datetime,turnstile
0,H007,R248,00-00-00,1 AV,2022-11-19,03:00:00,REGULAR,15874830,19176597,2022-11-19 03:00:00,H007-R248-00-00-00
1,H007,R248,00-00-01,1 AV,2022-11-19,03:00:00,REGULAR,61774939,39334861,2022-11-19 03:00:00,H007-R248-00-00-01
2,H007,R248,00-03-00,1 AV,2022-11-19,03:00:00,REGULAR,7462,42544,2022-11-19 03:00:00,H007-R248-00-03-00
3,H007,R248,00-03-01,1 AV,2022-11-19,03:00:00,REGULAR,2665210,1254388,2022-11-19 03:00:00,H007-R248-00-03-01
4,H007,R248,00-03-02,1 AV,2022-11-19,03:00:00,REGULAR,6768676,603401,2022-11-19 03:00:00,H007-R248-00-03-02


C:\Users\Didcvee\AppData\Local\Temp\ipykernel_4952\3844793025.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  station_grouped = df.groupby(['Station', pd.Grouper(key='datetime', freq='H')]).sum().reset_index()


TypeError: unsupported operand type(s) for +: 'datetime.date' and 'datetime.date'

In [12]:
import gradio as gr
import matplotlib.pyplot as plt

In [43]:
!pip install gradio


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [53]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd



# Преобразование данных
df['datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'])
df['entries_diff'] = df.groupby('turnstile')['Entries'].diff().fillna(0)
df['exits_diff'] = df.groupby('turnstile')['Exits'].diff().fillna(0)
df['traffic'] = df['entries_diff'] + df['exits_diff']

# Создание приложения Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Анализ данных станции метро"),
    html.Label("Выберите станцию:"),
    dcc.Dropdown(
        id='station-dropdown',
        options=[{'label': station, 'value': station} for station in df['Station'].unique()],
        value=df['Station'].unique()[0]
    ),
    html.Label("Выберите временной промежуток:"),
    dcc.DatePickerRange(
        id='date-picker',
        start_date=df['datetime'].min().date(),
        end_date=df['datetime'].max().date()
    ),
    dcc.Graph(id='traffic-graph'),
    dcc.Graph(id='capacity-graph'),
    dcc.Graph(id='top-stations-graph'),
    dcc.Graph(id='average-passengers-graph')
])

@app.callback(
    [Output('traffic-graph', 'figure'),
     Output('capacity-graph', 'figure'),
     Output('top-stations-graph', 'figure'),
     Output('average-passengers-graph', 'figure')],
    [Input('station-dropdown', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date')]
)
def update_graphs(selected_station, start_date, end_date):
    filtered_df = df[(df['Station'] == selected_station) & 
                     (df['datetime'] >= pd.to_datetime(start_date)) & 
                     (df['datetime'] <= pd.to_datetime(end_date))]

    # 1. Загруженность станции
    max_traffic = df['traffic'].max()
    filtered_df['load_percent'] = (filtered_df['traffic'] / max_traffic) * 100
    load_fig = px.line(filtered_df, x='datetime', y='load_percent', title='Загруженность станции (%)')

    # 2. Пропускная способность
    capacity_fig = px.line(filtered_df, x='datetime', y='traffic', title='Пропускная способность станции')

    # 3. Топ загруженных станций
    top_stations = df.groupby('Station')['traffic'].sum().reset_index()
    top_stations = top_stations.sort_values(by='traffic', ascending=False).head(10)
    top_stations_fig = px.bar(top_stations, x='Station', y='traffic', title='Топ загруженных станций')

    # 4. Среднее количество пассажиров
    avg_passengers = filtered_df['traffic'].mean()
    avg_passengers_fig = px.line(filtered_df, x='datetime', y='traffic', title=f'Среднее количество пассажиров: {avg_passengers:.2f}')

    return load_fig, capacity_fig, top_stations_fig, avg_passengers_fig

if __name__ == '__main__':
    app.run_server(debug=True)